In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
today

datetime.date(2022, 1, 17)

### Tables in the process

In [51]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
    
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'SHARES':'{:,}',    
    'Q4':'{:.4f}','Q3':'{:.4f}','Q2':'{:.4f}','Q1':'{:.4f}','DIVIDEND':'{:.4f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
}

In [9]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,KBANK,196,2021-11-30,800,136.88,905,1062,2022-01-17,144.50,239.29,16.75,"115,343.96",48,"5,597.42",5.10%,38.78%,95,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [10]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)

sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1062



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,KBANK,1062,196,2021-11-30,800,136.88,226.67,15.87,Sold,"109,746.54",DTD,None


In [11]:
transaction, name, qty, sell_price, buy_price

('S', 'KBANK', -800, 144.5, 136.88)

### In case of buy, retreive buy id from latest buy record (id)

In [35]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,RCL,1100,192,2022-01-17,1500,49.0,152.15,10.65,Active,73662.8,DTD,None


In [36]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'RCL', 1500, 49.0)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [12]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'KBANK'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,KBANK,2021-11-29,1600.0,136.5,0.0,0.0,1,3.0,4,A4


In [13]:
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -800
WHERE name = 'KBANK'



In [14]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [15]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy


SELECT * 
FROM buy 
WHERE name = 'KBANK'



,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,KBANK,2021-11-29,800.0,136.5,0.0,0.0,1,3.0,4,A4


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [23]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'KBANK'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,KBANK,2.5000,0.0000,0.5000,0.0000,3.0000,"1,600",2022-05-07,2022-05-07,,0


In [24]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + -800
WHERE name = 'KBANK'



In [25]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [27]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'KBANK'



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,KBANK,2.5000,0.0000,0.5000,0.0000,3.0000,800,2022-05-07,2022-05-07,,0


### If shares become zero and no longer wants to keep dividend record

In [ ]:
sqlDel = """
DELETE FROM dividend
WHERE name = '%s' AND shares = 0
"""
sqlDel = sqlDel % name
print(sqlDel)

In [ ]:
rp = const.execute(sqlDel)
rp.rowcount

### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [30]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'KBANK'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,637,KBANK,153.00,101.00,I,0.00,150.50,"3,860.83",1.70,136.50,800,-17,19,"1,600",0,0,10pct,SET50


In [31]:
sell_target = 150.5 # must calculate manually beforehand
#reason = "15pct"

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
sell_target = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_target, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + -800,
sell_target = 150.5
WHERE name = 'KBANK'



In [32]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [33]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'KBANK'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,637,KBANK,153.00,101.00,I,0.00,150.50,"3,860.83",1.70,136.50,800,-17,19,800,0,0,10pct,SET50


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [42]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy['volbuy'] = buy['volbuy'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'RCL'



,name,date,volbuy,price,active,dividend,period,grade
0,RCL,2021-11-08,6000,44.5,1,3.5,3,A4


### In case there is the same stock name in port

In [39]:
new_cost = 44.50 # new average cost must be calculated manually beforehand

sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 1500, price = 44.5
WHERE name = 'RCL'



In [40]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [43]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy['volbuy'] = buy['volbuy'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'RCL'



,name,date,volbuy,price,active,dividend,period,grade
0,RCL,2021-11-08,6000,44.5,1,3.5,3,A4


### In case of totally new stock

In [ ]:
sqlIns = """
INSERT INTO buy 
VALUES('LPF','2022-01-14',10000,14,0,0,1,0,'4','A3')
"""
print(sqlIns)

In [ ]:
#rp = const.execute(sqlIns)
rp.rowcount

### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [45]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'RCL' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,RCL,0.5000,1.5000,1.5000,0.0000,3.5000,"4,500",2021-12-03,2021-12-03,,0


In [46]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 1500
WHERE name = 'RCL'



In [47]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [48]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend['SHARES'] = dividend['SHARES'].astype('int64')
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'RCL' 



,NAME,Q4,Q3,Q2,Q1,DIVIDEND,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,RCL,0.5000,1.5000,1.5000,0.0000,3.5000,"6,000",2021-12-03,2021-12-03,,0


### End of Update dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [49]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)


SELECT * 
FROM stocks 
WHERE name = 'RCL'



In [52]:
stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,662,RCL,67.75,17.40,B,50.00,0.00,591.82,0.64,43.00,"1,500",-16,26,"4,500",0,0,RU15pct,SET100


In [53]:
new_cost = 44.5
sell_target = 49
status = 'I'
reason = '10pct'
new_sale_qty = 2000
new_cost, sell_target, status, reason, new_sale_qty

(44.5, 49, 'I', '10pct', 2000)

In [54]:
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, sell_target, status, reason, new_sale_qty, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 1500, 
cost = 44.5, sell_target = 49, status = 'I', 
reason = '10pct', qty = 2000
WHERE name = 'RCL'



In [55]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [56]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'RCL'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,662,RCL,67.75,17.40,I,50.00,49.00,591.82,0.64,44.50,"2,000",-16,26,"6,000",0,0,10pct,SET100


### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process